<a href="https://colab.research.google.com/github/bellomusodiq/machine_learning/blob/master/text_generation_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.nn.utils import clip_grad_norm_

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class Dictionary(object):
  def __init__(self):
    self.word2idx = {}
    self.idx2word = {}
    self.idx = 0

  def add_word(self, word):
    if word not in self.word2idx:
      self.word2idx[word] = self.idx
      self.idx2word[self.idx] = word
      self.idx += 1

  def __len__(self):
    return len(self.word2idx)


In [0]:
class TextProcess(object):

  def __init__(self):
    self.dictionary = Dictionary()

  def get_data(self, path, batch_size=20):
    vocab = []
    with open(path, 'r') as f:
      tokens = 0
      for line in f:
        words = line.split() + ['<eos>']
        tokens += len(words)
        for word in words:
          self.dictionary.add_word(word)
          vocab.append(self.dictionary.word2idx[word])
    rep_tensor = torch.tensor(vocab, dtype=torch.int64)
    num_batches = rep_tensor.shape[0] // batch_size
    rep_tensor = rep_tensor[:num_batches*batch_size]
    rep_tensor = rep_tensor.view(batch_size, -1)
    return rep_tensor

In [0]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epoch = 200
batch_size = 20
timestep = 30
learning_rate = .002

In [0]:
corpus = TextProcess()

In [0]:
rep_tensor = corpus.get_data('drive/My Drive/alice.txt', batch_size).cuda()

In [8]:
rep_tensor.shape

torch.Size([20, 1484])

In [9]:
vocab_size = len(corpus.dictionary)
vocab_size

5290

In [10]:
num_batches = rep_tensor.shape[1] // timestep
num_batches

49

In [0]:
class TextGenerator(nn.Module):

  def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
    super(TextGenerator, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, h):
    x = self.embed(x)
    out, (h, c) = self.lstm(x, h)
    out = out.reshape(out.size(0)*out.size(1), out.size(2))
    out = self.linear(out)
    return out, (h, c)

In [0]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers).cuda()

In [13]:
print(model)

TextGenerator(
  (embed): Embedding(5290, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear(in_features=1024, out_features=5290, bias=True)
)


In [0]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epoch):

  states = (
      torch.zeros(num_layers, batch_size, hidden_size).cuda(),
      torch.zeros(num_layers, batch_size, hidden_size).cuda()
  )

  for i in range(0, rep_tensor.size(1) - timestep, timestep):
    inputs = rep_tensor[:, i:i+timestep].cuda()
    targets = rep_tensor[:, (i+1):(i+1)+timestep].cuda()

    outputs, _ = model(inputs, states)
    loss = loss_fn(outputs, targets.reshape(-1))

    optimizer.zero_grad()
    loss.backward()

    clip_grad_norm_(model.parameters(), .5)
    optimizer.step()

  print('Epoch {}/{}, Loss: {:.4f}'
  .format(epoch+1, num_epoch, loss.item()))

Epoch 1/200, Loss: 6.4910
Epoch 2/200, Loss: 5.6114
Epoch 3/200, Loss: 4.9870
Epoch 4/200, Loss: 4.4983
Epoch 5/200, Loss: 4.0650
Epoch 6/200, Loss: 3.4978
Epoch 7/200, Loss: 3.0938
Epoch 8/200, Loss: 2.6037
Epoch 9/200, Loss: 2.2076
Epoch 10/200, Loss: 1.8137
Epoch 11/200, Loss: 1.5500
Epoch 12/200, Loss: 1.1924
Epoch 13/200, Loss: 1.0207
Epoch 14/200, Loss: 0.6521
Epoch 15/200, Loss: 0.4460
Epoch 16/200, Loss: 0.2696
Epoch 17/200, Loss: 0.1584
Epoch 18/200, Loss: 0.0949
Epoch 19/200, Loss: 0.0756
Epoch 20/200, Loss: 0.0728
Epoch 21/200, Loss: 0.0671
Epoch 22/200, Loss: 0.0690
Epoch 23/200, Loss: 0.0644
Epoch 24/200, Loss: 0.0669
Epoch 25/200, Loss: 0.0628
Epoch 26/200, Loss: 0.0653
Epoch 27/200, Loss: 0.0619
Epoch 28/200, Loss: 0.0641
Epoch 29/200, Loss: 0.0612
Epoch 30/200, Loss: 0.0632
Epoch 31/200, Loss: 0.0606
Epoch 32/200, Loss: 0.0626
Epoch 33/200, Loss: 0.0601
Epoch 34/200, Loss: 0.0620
Epoch 35/200, Loss: 0.0597
Epoch 36/200, Loss: 0.0614
Epoch 37/200, Loss: 0.0593
Epoch 38/2

In [16]:
with torch.no_grad():
  with open('results.txt', 'w') as f:
    state = (
        torch.zeros(num_layers, 1, hidden_size).cuda(),
        torch.zeros(num_layers, 1, hidden_size).cuda(),
    )
    input = torch.randint(0, vocab_size, (1,)).long().unsqueeze(0)
    
    for i in range(500):
      output, _ = model(input.cuda(), state)
      prob = output.exp()
      word_id = torch.multinomial(prob, num_samples=1).item()
      input.fill_(word_id)

      word = corpus.dictionary.idx2word[word_id]
      word = '\n' if word == '<eos>' else word + ' '
      f.write(word)

      if (i+1) % 100 == 0:
        print('Sampled {}/{}'.format(i, 500))

Sampled 99/500
Sampled 199/500
Sampled 299/500
Sampled 399/500
Sampled 499/500


In [18]:
targets.shape

torch.Size([20, 30])